In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

from multimodal_transformers.data.load_data import (load_train_val_test_helper, 
                                                    load_data, 
                                                    load_data_into_folds,
                                                    load_data_from_folder)

from transformers import AutoTokenizer, EvalPrediction

import warnings
warnings.filterwarnings('ignore')

data_df = pd.read_csv('../data/clean_data/listings.csv')

data_df=data_df.dropna()

data_df.head()

import multimodal_transformers.data as d
help(d.load_data)

In [ ]:
train=data_df.iloc[:200]

test=data_df.iloc[21250:]

train.shape, test.shape

In [ ]:
text_cols = ['room_type', 'amenities']

label_col = 'price' 

categorical_cols = ['neighbourhood_cleansed', 'neighbourhood_group_cleansed']

numerical_cols = ['accommodates', 'bedrooms', 'beds']

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', truncation=True)

train_data = load_data(train, text_cols, tokenizer, categorical_cols=categorical_cols,
                       numerical_cols=numerical_cols, sep_text_token_str=tokenizer.sep_token,
                       label_col=label_col)


test_data = load_data(test, text_cols, tokenizer, categorical_cols=categorical_cols,
                       numerical_cols=numerical_cols, sep_text_token_str=tokenizer.sep_token,
                       label_col=label_col)

In [ ]:
train_df, val_df, test_df=load_train_val_test_helper(train_df=train, 
                           val_df=test, 
                           test_df=test,
                           label_col=label_col,
                           text_cols=text_cols, 
                           tokenizer=tokenizer, 
                           categorical_cols=categorical_cols,
                           numerical_cols=numerical_cols, 
                           sep_text_token_str=tokenizer.sep_token)

In [ ]:
from multimodal_transformers.model import AutoModelWithTabular, TabularConfig
from transformers import AutoConfig

config=AutoConfig.from_pretrained('bert-base-uncased')

tabular_config=TabularConfig(
    num_labels=1,
    cat_feat_dim=train_data.cat_feats.shape[1],
    numerical_feat_dim=train_data.numerical_feats.shape[1],
    combine_feat_method='weighted_feature_sum_on_transformer_cat_and_numerical_feats',
)
config.tabular_config = tabular_config

model = AutoModelWithTabular.from_pretrained('bert-base-uncased', config=config)

In [ ]:
import torch

device=torch.device('mps')  # mps neural engine M1, cuda o cpu

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error


def metrics(y_test, y_pred):
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    
    return {'mse': mse,
            'rmse': rmse,
            'mae': mae}

In [ ]:
from transformers import Trainer, TrainingArguments

args=TrainingArguments(output_dir='./logs/model_name',
                       logging_dir='./logs/runs',
                       overwrite_output_dir=True,
                       per_device_train_batch_size=32,
                       num_train_epochs=0.01,
                       evaluate_during_training=True,
                       logging_steps=25)



trainer=Trainer(model=model.to(device), 
                args=args, 
                train_dataset=train_df,
                eval_dataset=val_df,
                #compute_metrics=metrics
               )

In [ ]:
%%time

trainer.train()

In [ ]:
%%time

y_pred=trainer.predict(test_dataset=test_df).predictions

In [ ]:
trainer.evaluate(train_data)

In [ ]:
help(Trainer)

In [ ]:
from torch.utils.data import Subset

Subset(train_data, [0, 1, 2])

In [ ]:
help(train_data)

In [ ]:
y_test=train.price.values

metrics(y_test, y_pred)

In [ ]:
trainer.evaluate(Subset(train_data, [0, 1, 2]))